# Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [2]:
import itertools
# Данная библиотека понадобится нам, чтобы обработать разметку
# !pip install xmltodict pytorch-lightning

Скачаем данные

In [3]:
# !wget --quiet --load-cookies / tmp / cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip & & rm -rf / tmp / cookies.txt
# !unzip -q data.zip
# !rm data.zip
# !ls -l

Посмотрим как выглядит один из файлов разметки

In [4]:
# !cat data/train/apple_3.xml

## Релизуйте выборку для YoloV1 - 2 балла

In [5]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [6]:
from math import floor

class2tag = {'apple': 0, 'orange': 1, 'banana': 2}


class FruitDataset(Dataset):
    def __init__(self, data_dir, S=7, B=2, C=3, transforms=None):
        self.S = S
        self.B = B
        self.C = C

        self.image_paths = sorted(glob.glob(os.path.join(data_dir, '*.jpg')))
        self.box_paths = sorted(glob.glob(os.path.join(data_dir, '*.xml')))

        assert len(self.image_paths) == len(self.box_paths)

        self.transforms = transforms

    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert('RGB'))
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx])

        # Переводим координаты боксов в формат YOLO
        boxes = list(map(lambda coords: self.__convert_to_yolo_box_params(coords,
                                                                          image.shape[1],
                                                                          image.shape[0]),
                         boxes))

        # Применяем преобразования
        if self.transforms:
            transformed = self.transforms(image=image, bboxes=boxes, class_labels=class_labels)
            image = transformed['image']
            boxes = transformed['bboxes']
            class_labels = transformed['class_labels']

        # Делаем не совсем корректное предположение, что в одной ячейке может быть только один объект
        target_tensor = torch.zeros((self.S, self.S, 5 * self.B + self.C))
        for box, label in zip(boxes, class_labels):
            x, y, w, h = box

            # Получаем номер ячейки
            xn, yn = int(x * self.S), int(y * self.S)

            # Пересчитываем координаты относительно ячейки
            x, y = self.S * x - xn, self.S * y - yn
            w, h = w * self.S, h * self.S

            # Сохраняем в формате [class1, class2, class3, confidence, x, y, w, h, ...]
            target_tensor[xn][yn][label] = 1
            target_tensor[xn][yn][self.C:self.C + 5] = torch.Tensor([1, x, y, w, h])

        # Переводим картинку в тензор
        image = torch.from_numpy(image).float().permute((2, 0, 1))
        return image, target_tensor

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str):
        """
        Метод, который считает и распарсит (с помощью xmltodict) переданный xml
        файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
        и название класса обьекта в каждом прямоугольнике

        Обратите внимание, что обьектов может быть как несколько, так и один единственный
        """
        boxes = []
        class_labels = []

        with open(xml_filename) as f:
            contents = xmltodict.parse(f.read())['annotation']['object']

        if isinstance(contents, dict):
            contents = [contents]
        for obj in contents:
            class_labels.append(class2tag[obj['name']])
            boxes.append(list(map(int, obj['bndbox'].values())))

        return boxes, class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
        """
        Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].

        Обратите внимание, что параметры [x_center, y_center, width, height] - это
        относительные значение в отрезке [0, 1]

        :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
        :param: im_w - ширина исходного изображения
        :param: im_h - высота исходного изображения

        :return: координаты коробки в формате [x_center, y_center, width, height]
        """
        ans = []

        ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)  # x_center
        ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center

        ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)  # width
        ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height
        return ans


In [7]:
SIZE = 448

train_transform = A.Compose([
    A.SmallestMaxSize(SIZE),
    A.RandomCrop(width=SIZE, height=SIZE),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2)],  ## YOUR CODE
    bbox_params=A.BboxParams(format='yolo',
                             label_fields=['class_labels']))
test_transform = A.Compose([
    A.SmallestMaxSize(SIZE),
    A.CenterCrop(width=SIZE, height=SIZE)],  ## YOUR CODE
    bbox_params=A.BboxParams(format='yolo',
                             label_fields=['class_labels']))

In [8]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
)

val_dataset = FruitDataset(
    transforms=test_transform,
    data_dir="./data/test"
)

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[0][0], torch.Tensor)
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [9]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=4,
    shuffle=True)

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=4,
    shuffle=False
)

Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [10]:
def intersection_over_union(pr_bboxes, gt_bboxes) -> torch.Tensor:
    pr_x, pr_y = pr_bboxes[..., 0], pr_bboxes[..., 1]
    pr_w, pr_h = pr_bboxes[..., 2], pr_bboxes[..., 3]

    pr_xmin, pr_xmax = pr_x - pr_w / 2, pr_x + pr_w / 2
    pr_ymin, pr_ymax = pr_y - pr_h / 2, pr_y + pr_h / 2

    gt_x, gt_y = gt_bboxes[..., 0], gt_bboxes[..., 1]
    gt_w, gt_h = gt_bboxes[..., 2], gt_bboxes[..., 3]

    gt_xmin, gt_xmax = gt_x - gt_w / 2, gt_x + gt_w / 2
    gt_ymin, gt_ymax = gt_y - gt_h / 2, gt_y + gt_h / 2

    it_xmin, it_xmax = torch.min(pr_xmin, gt_xmin), torch.max(pr_xmax, gt_xmax)
    it_ymin, it_ymax = torch.min(pr_ymin, gt_ymin), torch.max(pr_ymax, gt_ymax)

    it_area = (it_xmax - it_xmin).clamp(0) * (it_ymin - it_ymax).clamp(0)
    pr_area = (pr_xmax - pr_xmin) * (pr_ymax - pr_ymin)
    gt_area = (gt_xmax - gt_xmin) * (gt_ymax - gt_ymin)

    return it_area / (pr_area + gt_area - it_area + 1e-6)

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [11]:
CONV_ARCH = [
    (7, 64, 2, 3, True),

    (3, 192, 1, 1, True),

    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1, True),

    [4, (1, 256, 1, 0), (3, 512, 1, 1)],
    (1, 512, 1, 0),
    (3, 1024, 1, 1, True),

    [2, (1, 512, 1, 0), (3, 1024, 1, 1)],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),

    [2, (3, 1024, 1, 1)]
]

In [12]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool: bool = False, **kwargs):
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2, stride=2)

    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x


class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """

        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C

        layers = []
        in_channels = 3

        # Сверточные слои
        for convLayer in CONV_ARCH:
            if isinstance(convLayer, list):
                n, *l = convLayer
            else:
                n, l = 1, [convLayer]

            block = []
            for layer in l:
                kernel_size, out_channels, stride, padding, *pool = layer
                block.append(CNNBlock(in_channels=in_channels,
                                      out_channels=out_channels,
                                      kernel_size=kernel_size,
                                      stride=stride,
                                      padding=padding,
                                      is_max_pool=pool))
                in_channels = out_channels
            layers.extend(block * n)

        # Полносвязные слои
        layers += [
            nn.Flatten(),
            nn.Linear(7 * 7 * 1024, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, self.S * self.S * (self.C + 5 * self.B))
        ]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


# Убедитесь на одном изображении, что предсказания вашей модели имеют верное количество значений

temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

testing_image = train_dataset[0][0].unsqueeze(dim=0)
temp_model.eval()
assert temp_model(testing_image).reshape(-1).shape[0] == expected_output_shape

## Реализуйте YoloLoss - 3 балла

In [25]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """

        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, prediction, target):
        prediction = prediction.reshape(self.S, self.S, self.C + self.B * 5)

        temp = [intersection_over_union(prediction[..., self.C + 1 + 5 * t:self.C + 5 + 5 * t],
                                        target[..., self.C + 1:self.C + 5])
                for t in range(self.B)]
        ious = torch.stack(temp).permute((1, 2, 0))
        max_ious, _ = torch.max(ious, dim=2)
        temp = [torch.stack([ious[i][j] == max_ious[i][j]
                 for i in range(self.S)])
                for j in range(self.S)]
        is_best_bbox = torch.stack(temp)

        I_i = target[..., self.C].unsqueeze(dim=2)
        I_ij = I_i * is_best_bbox

        term1 = sum(I_ij[...,t].unsqueeze(dim=2) * (prediction[...,self.C +1 + 5*t:self.C+3+5*t] - target[..., self.C+1:self.C+3])**2
                    for t in range(self.B))

        term2 = sum(I_ij[...,t].unsqueeze(dim=2) * (torch.sqrt(prediction[...,self.C +3 + 5*t:self.C+5+5*t]) - torch.sqrt(target[..., self.C+3:self.C+5]))**2
                    for t in range(self.B))

        term3 = sum(I_ij[...,t].unsqueeze(dim=2) * (prediction[...,self.C + 5*t] - target[..., self.C])**2
                    for t in range(self.B))

        term4 = sum((1 - I_ij[...,t]) * (prediction[...,self.C + 5*t] - target[..., self.C])**2
                    for t in range(self.B))

        term5 = sum(I_i * (prediction[..., :self.C] - target[..., :self.C])**2)

        return self.lambda_coord * (term1.sum() + term2.sum()) + term3.sum() + self.lambda_noobj * term4.sum() + term5.sum()

loss = YoloLoss()
model = YOLO()
image, label = train_dataloader.dataset[0]
image = image.unsqueeze(dim=0)
loss(model(image), label)

tensor(nan, grad_fn=<AddBackward0>)

## Реализуйте дополнительные функции из статьи - 2 балла

In [ ]:
def non_max_suppression(bboxes, iou_threshold, threshold):
    ## YOUR CODE
    pass


def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5):
    ## YOUR CODE
    pass


def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    ## YOUR CODE
    return all_pred_boxes, all_true_boxes

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        ## YOUR CODE
        pass

    def validation_step(self, val_batch, batch_idx) -> None:
        ## YOUR CODE
        pass

In [ ]:
model =  # YOUR CODE
n_epochs =  # YOUR CODE

yolo_learner = YOLOLearner(...)  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image